Так как конечноая цель EDA не до конца ясна, проведу его в обобщенном виде и не до конца(почищу данные, может подменю какие-то характеристики/нормализую)

In [124]:
import pandas as pd
import re
import numpy as np

грузим датасет

In [125]:
df=pd.read_csv("before_eda.csv")

посмотрим что в нем

In [126]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2995 entries, 0 to 2994
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  2995 non-null   int64  
 1   name                        2995 non-null   object 
 2   description                 2995 non-null   object 
 3   price                       2072 non-null   float64
 4   reviews_count               2995 non-null   int64  
 5   rating                      2995 non-null   float64
 6   Тип                         2995 non-null   object 
 7   Тип пластика для 3D печати  2995 non-null   object 
 8   Диаметр, мм                 2899 non-null   float64
 9   Размеры, мм                 2346 non-null   object 
 10  Вес, кг                     2805 non-null   float64
 11  Технология 3D печати        2794 non-null   object 
 12  Твердость по Шору           1337 non-null   object 
 13  Объем, л                    653 n

посмотрим колонки где значений совсем мало

In [127]:
filled_value_counts = df.count()
filled_value_counts_sorted = filled_value_counts.sort_values()
print(filled_value_counts_sorted)

Бренд                            1
Цвет:                            1
Размеры, мм:                     2
Количество, шт                 169
Объем, л                       653
Длина, см                     1183
Твердость по Шору             1337
Гарантийный срок              1666
price                         2072
Размеры, мм                   2346
Технология 3D печати          2794
Вес, кг                       2805
Вес товара, г                 2852
Масса, кг                     2873
Диаметр, мм                   2899
Цвет                          2927
Страна-изготовитель           2928
Тип пластика для 3D печати    2995
Тип                           2995
rating                        2995
reviews_count                 2995
description                   2995
name                          2995
Unnamed: 0                    2995
dtype: int64


Удалим те параметры у которых заполненность менее 30%, а так же удалим объем в литрах. Это пластик и он продается в бухтах на вес(у которого есть прямая зависимость с длиной, поэтому её тоже можно удалить)

In [128]:
features_with_na=df.isna().sum()[df.isna().sum()>0] / df.shape[0]
list_to_drop=features_with_na[features_with_na > 0.7].index.tolist()
df.drop(list_to_drop, axis=1, inplace=True)

поудаляем проблемные вхождения(видимо что-то шло не так, цена явно должна быть у всех, так что откинем сразу те в которых цены нет), а так же сюда вошел индекс, он нам не нужен(видимо  в процессе выгрузки где-то его впустили=), удалим и его.
Опять же это пластик, который идет на вес, удалим длину в см

In [129]:
df = df.dropna(subset=['price'])
list_to_drop=['Unnamed: 0','Длина, см']
df.drop(list_to_drop, axis=1, inplace=True)

посмотрим уникальные значения диаметра. есть мнение что оно будет почти одним и тем же

In [130]:
df['Диаметр, мм'].value_counts()

Диаметр, мм
1.75    1952
2.85      16
1.00       7
1.80       6
2.75       1
9.75       1
3.75       1
3.00       1
Name: count, dtype: int64

видим что почти везде он стандартный 1.75. удалим все вхождения, где иначе. и удалим колонку,  так как стандарт пластика 1.75

In [131]:
df = df[df['Диаметр, мм'] == 1.75]
del df['Диаметр, мм']

пластик без веса нас явно не интересует, удалим и эти пустые вхождения, а так же удалим колонку 'Масса, кг','Вес, кг' так как не понятно зачем данные дублируются(физически это конечно разные понятия=), но не для покупателя)

In [132]:
df = df.dropna(subset=['Вес товара, г'])
list_to_drop=['Масса, кг','Вес, кг']
df.drop(list_to_drop, axis=1, inplace=True)

после всей этой чистки видим что цвет,страна и технология заполнены почти везде. Думаю откинуть оставшиеся мелочи можно сразу

In [133]:
df = df.dropna(subset=['Технология 3D печати','Цвет','Страна-изготовитель'])

в принципе уже неплохая выборка получилась. гарантийный срок если отсутствует - значит скорей всего его нет, заполняем 0-ми, а остальные значения мапим

In [134]:
df['Гарантийный срок'].value_counts()

Гарантийный срок
12 месяцев                                 345
365                                        105
1 год                                      101
1 год с даты производства товара            98
20.0                                        58
24 месяца                                   54
1                                           49
730                                         28
12 мес                                      26
2 года                                      19
547                                         18
14 дней                                     16
-                                           10
30 дней                                     10
1.0                                          9
6 месяцев                                    7
6 мес                                        6
нет                                          5
12                                           4
1 год.                                       4
36 месяцев                                 

In [135]:
mapping = {
    '12 месяцев': 365,
    '365': 365,
    '1 год': 365,
    '1 год с даты производства товара': 365,
    '20.0': 600,
    '24 месяца': 730,
    '1': 365,
    '730': 730,
    '12 мес': 365,
    '2 года': 730,
    '547': 547,
    '14 дней': 14,
    '-': 0,
    '30 дней': 30,
    '1.0': 365,
    '6 месяцев': 180,
    '6 мес': 180,
    'нет': 0,
    '12': 365,
    '1 год.': 365,
    '36 месяцев': 432,
    '18 месяцев': 547,
    '12 мес.': 365,
    '3 месяца': 90,
    'не ограничен': 0,
    '540': 540,
    'Производитель не предоставляет гарантию': 0,
    '2': 365,
    '2 недели': 14,
    '90': 90,
    '5 лет': 1825
}
df['Гарантийный срок'] = df['Гарантийный срок'].map(mapping)
df['Гарантийный срок'] = df['Гарантийный срок'].fillna(0)

посмотрим на размеры

In [136]:
pd.options.display.max_rows = 1000
df['Размеры, мм'].value_counts()

Размеры, мм
200x200x80                     157
20 х 20 х 8 см (0,0032 м3)     145
210х230х80                      82
220х230х80                      75
200х200                         70
190 x 190 x 55                  62
200x200x70                      60
200 x 200 x 80                  58
230x230x80                      51
220х220х80                      37
200х200х65                      35
280x280x130                     35
220х200х70                      31
198x198x65                      29
250x250x80                      28
1,75                            25
30 х 30 х 16 см (0,0144 м3)     24
220х230х25                      20
220x210x75                      20
100x100x10                      17
210x210x80                      16
200X200X64                      16
190х190х55                      14
200*200*64                      14
220x200x100                     14
200х200х58                      13
200х200х60                      13
250х250х100                     13
203x203x

видим разнобой размерностей. возьмем да и переведем все в объем

In [137]:
df['Размеры, мм'] = df['Размеры, мм'].fillna('0')
def calculate_volume(value):
    values = re.findall(r'\d+', value)
    
    if len(values) >= 3:
        if "см" in value:
            volume = (int(values[0]) * int(values[1]) * int(values[2])) / 1000000
        elif any(int(val) < 10 for val in values):
            volume = (int(values[0]) * int(values[1]) * int(values[2])) / 1000000
        else:
            volume = int(values[0]) * int(values[1]) * int(values[2]) / 1000000000
    else: # Если указано менее 3 размерностей
        volume = 0  
    
    return round(volume, 6)
df['Объем (м³)'] = df['Размеры, мм'].apply(calculate_volume)
df['Объем (м³)'].replace(0, np.nan, inplace=True)
del df['Размеры, мм']

In [138]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1783 entries, 0 to 2994
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   name                        1783 non-null   object 
 1   description                 1783 non-null   object 
 2   price                       1783 non-null   float64
 3   reviews_count               1783 non-null   int64  
 4   rating                      1783 non-null   float64
 5   Тип                         1783 non-null   object 
 6   Тип пластика для 3D печати  1783 non-null   object 
 7   Технология 3D печати        1783 non-null   object 
 8   Твердость по Шору           866 non-null    object 
 9   Вес товара, г               1783 non-null   float64
 10  Цвет                        1783 non-null   object 
 11  Страна-изготовитель         1783 non-null   object 
 12  Гарантийный срок            1783 non-null   float64
 13  Объем (м³)                  1338 non-n

по сути, по хорошему нужно выяснить так же а как же лучше заполнить на самом деле объем вместо 0(я пока чтобы в дальнейшем не забыть - вернул NaN). Но так как не ясна конечная цель - на текущий момент я не стал этого делать.
еще остается так же не заполненная до конца твердость п Шору, которую с большей доли вероятности пока откидывать нельзя, но было бы тоже неплохо заполнить чем-то (медианами, квантилями, чем-то определенным, etc)
и быть может нужно было бы избавиться от словарных значений в цвете, стране, технологиях... просто закодировав их в то что нужно. но это тоже не выполнял.
так же вероятно нужно нормализовать данные, там где у нас числа.

На мой взгляд - данный eda notebook максимально подготовил данные из полученного в результате парсинга dataset для дальнешего использования в условиях неопределенности задачи